In [12]:
import openrouteservice
import folium
from IPython.display import display
from itertools import permutations

# Initialize OpenRouteService client with API key
client = openrouteservice.Client(key='5b3ce3597851110001cf6248940c956ef1b941d7aa32623ece4bfe7c')

def get_route_between_points(start, end):
    """Fetches the route between two points using OpenRouteService directions API."""
    route = client.directions(
        coordinates=[(start[1], start[0]), (end[1], end[0])],
        profile='driving-car',  # Ensures the route follows roads
        format='geojson'
    )
    return route

def calculate_route_brute_force(locations, distance_matrix): 
    """Calculate the shortest route using brute force."""
    num_locations = len(locations)
    shortest_distance = float('inf')
    best_route = None

    # Generate all permutations of locations except the starting point
    for perm in permutations(range(1, num_locations)):
        # Calculate the total distance for this permutation
        total_distance = distance_matrix[0][perm[0]]  # Distance from start to first point
        for i in range(1, len(perm)):
            total_distance += distance_matrix[perm[i-1]][perm[i]]
        total_distance += distance_matrix[perm[-1]][0]  # Return to start

        # If this route is shorter, remember it
        if total_distance < shortest_distance:
            shortest_distance = total_distance
            best_route = perm

    return best_route, shortest_distance

def get_distance_matrix(locations):
    """Fetches the distance matrix from OpenRouteService for given locations."""
    coords = [[loc[1], loc[0]] for loc in locations]  # Swap (lat, lon) to (lon, lat) format
    matrix = client.distance_matrix(coords, metrics=['distance'])
    return matrix['distances']

def plot_route_on_map(locations, best_route):
    """Plots the optimal route on a map using folium with real roads."""
    # Start by creating a map centered around the first location
    m = folium.Map(location=locations[0], zoom_start=13)

    # Add markers for all locations
    folium.Marker(locations[0], popup="Start/End", icon=folium.Icon(color='green')).add_to(m)
    for i, loc in enumerate(best_route, start=1):
        folium.Marker(locations[loc], popup=f"Location {i}").add_to(m)

    # Draw routes following real roads
    route_coords = [locations[0]] + [locations[i] for i in best_route] + [locations[0]]
    
    for i in range(len(route_coords) - 1):
        route = get_route_between_points(route_coords[i], route_coords[i + 1])
        geometry = route['features'][0]['geometry']['coordinates']
        folium.PolyLine([(coord[1], coord[0]) for coord in geometry], color="blue", weight=2.5, opacity=1).add_to(m)

    # Display the map inline
    display(m)

def main():
    # Get user's name
    user_name = input("Enter your name: ")

    # Get number of locations to visit
    n = int(input("Enter the number of locations to visit (including start/end point): "))

    # Collect locations (latitude and longitude)
    locations = []
    for i in range(n):
        lat = float(input(f"Enter the latitude for location {i+1}: "))
        lon = float(input(f"Enter the longitude for location {i+1}: "))
        locations.append((lat, lon))

    # Fetch the distance matrix
    print("Fetching distance matrix from OpenRouteService...")
    distance_matrix = get_distance_matrix(locations)

    # Calculate the optimal route using brute force algorithm
    best_route, shortest_distance = calculate_route_brute_force(locations, distance_matrix)

    # Print the optimal route
    print(f"\nHello {user_name}, the optimal route for visiting your locations is:")
    print(locations[0], " -> ", end="")
    for i in best_route:
        print(locations[i], " -> ", end="")
    print(locations[0])

    print(f"Total distance: {shortest_distance / 1000:.2f} km")

    # Plot the route on a map with real roads
    plot_route_on_map(locations, best_route)

if __name__ == "__main__":
    main()


Enter your name:  KS
Enter the number of locations to visit (including start/end point):  2
Enter the latitude for location 1:  34.058126497020844
Enter the longitude for location 1:  -117.82142947952937
Enter the latitude for location 2:  34.0603068980357
Enter the longitude for location 2:   -117.82115454884296


Fetching distance matrix from OpenRouteService...

Hello KS, the optimal route for visiting your locations is:
(34.058126497020844, -117.82142947952937)  -> (34.0603068980357, -117.82115454884296)  -> (34.058126497020844, -117.82142947952937)
Total distance: 2.73 km
